# Superradiant Decay with Embedded TLS Molecules

"
            "This tutorial reproduces the superradiant relaxation benchmark using purely"
            " embedded (non-socket) TLS drivers. All molecules couple to the same Meep"
            " simulation, share identical spatial profiles, and start with a small"
            " excited-state population.

**Prerequisites**

"
            "- `pymeep` / `meep` built with MPI support (optional but recommended)
"
            "- `matplotlib` for visualization

"
            "The parameters mirror those used in `tests/test_tls/test_meep_2d_tlsmolecule_n_relaxation.py`.

## 1. Define the experiment parameters

"
            "We specify the number of TLS molecules, their resonance frequency, dipole"
            " moment scaling, and the Meep grid settings. Setting `time_units_fs` to 0.1 fs"
            " reproduces the convention assumed by the TLS driver.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import maxwelllink as mxl

try:  # noqa: SIM105 - explicit tutorial hint
    import meep as mp
except ImportError as exc:  # pragma: no cover - docs hint
    raise RuntimeError('Meep is required for this tutorial. Install it via conda-forge.') from exc

N_TLS = 10
OMEGA_AU = 0.242
MU12_SINGLE_AU = 0.1  # single-molecule dipole moment (Meep units)
TIME_UNITS_FS = 0.1
RESOLUTION = 10
SIGMA = 0.1
CELL_SIZE = mp.Vector3(8, 8, 0)
PML_LAYERS = [mp.PML(3.0)]

## 2. Instantiate the TLS molecules

"
            "Each TLS is created in non-socket mode (`driver='tls'`). We divide the dipole"
            " moment by √N so that the collective coupling strength remains constant as"
            " we add more emitters, a standard superradiance scaling.

In [ ]:
molecules = []
for _ in range(N_TLS):
    molecules.append(
        mxl.Molecule(
            driver='tls',
            resolution=RESOLUTION,
            center=mp.Vector3(0, 0, 0),
            size=mp.Vector3(1, 1, 1),
            sigma=SIGMA,
            dimensions=2,
            driver_kwargs=dict(
                omega=OMEGA_AU,
                mu12=187 / np.sqrt(N_TLS),
                orientation=2,
                pe_initial=1e-2,
            ),
        )
    )

print(f'Configured {len(molecules)} TLS molecules.')

## 3. Build the Meep simulation

"
            "`MeepSimulation` automatically inserts the coupling step for embedded drivers."
            " No external `SocketHub` is required in this workflow.

In [ ]:
sim = mxl.MeepSimulation(
    molecules=molecules,
    cell_size=CELL_SIZE,
    boundary_layers=PML_LAYERS,
    resolution=RESOLUTION,
    time_units_fs=TIME_UNITS_FS,
)

print('Simulation ready — starting propagation...')

## 4. Propagate the coupled system

"
            "We evolve the simulation for 90 Meep time units (9 fs with `time_units_fs = 0.1`)."
            " The TLS drivers store their populations and additional diagnostics in"
            " `additional_data_history`.

In [ ]:
sim.run(until=90)

population = np.array([entry['Pe'] for entry in molecules[0].additional_data_history])
time_au = np.array([entry['time_au'] for entry in molecules[0].additional_data_history])
print(f'Collected {population.size} samples from each TLS driver.')

## 5. Validate the superradiant decay law

"
            "In two dimensions the spontaneous-emission rate scales linearly with the number"
            " of emitters. Using the analytical expression, we compare the simulated population"
            " to the expected collective decay and quote the relative error.

In [ ]:
time_meep = time_au * 0.02418884254 / TIME_UNITS_FS
initial_population = population[0]

mu_single = MU12_SINGLE_AU / np.sqrt(N_TLS)
frequency_meep = 1.0  # corresponds to omega=0.242 a.u. when TIME_UNITS_FS = 0.1

single_gamma = mu_single**2 * frequency_meep**2 / 2.0
collective_gamma = single_gamma * N_TLS
reference = np.exp(-time_meep * collective_gamma) / (
    np.exp(-time_meep * collective_gamma) + (1.0 - initial_population) / initial_population
)

std_rel = np.std(population - reference) / initial_population
max_rel = np.max(np.abs(population - reference)) / initial_population
print(f'std_rel = {std_rel:.3e}, max_rel = {max_rel:.3e}')

plt.figure(figsize=(6, 4))
plt.plot(time_meep, population, label='Simulation')
plt.plot(time_meep, reference, '--', label='Analytical reference')
plt.xlabel('time (Meep units)')
plt.ylabel('excited-state population')
plt.legend()
plt.tight_layout()
plt.show()

### Wrap-up

"
            "The embedded TLS workflow reproduces the collective enhancement of the decay"
            " rate without relying on socket-based drivers, making it ideal for lightweight"
            " regression tests and rapid prototyping.